# Load data

In [ ]:
from cache_utils import saved_data

In [ ]:
items_id_all = list(saved_data['items_dict'].keys())
items_id = items_id_all[:5]
print(len(items_id_all))
print(saved_data['items_dict'][items_id[0]]['item']['name']) #.keys())

In [ ]:
items_id[0]

In [ ]:
# list(saved_data['items_dict'][items_id[0]]['item'].keys())

In [ ]:
# saved_data['items_dict'][items_id[0]]

# Insert to ES as documents

In [ ]:
from elasticsearch import Elasticsearch
es=Elasticsearch([{'host':'localhost','port':9200}])

In [ ]:
from flatten_dict import flatten

In [ ]:
# _id
_id = (2153259823, 27618473)

In [ ]:
# for i, _id in enumerate(items_id_all[i:]):
if True:
    item = flatten(saved_data['items_dict'][_id])
    rem_l = []
    for k in item:
        v = item[k]
        if type(v) is dict or type(v) is list:
            rem_l.append(k)

    for k in rem_l:
        item.pop(k)

    new_item = {}
    for k in item:
        new_item["_".join(map(str, k))] = item[k]

    res=es.index(index='shopee',id=str(_id),body=new_item)

In [ ]:
res=es.get(index='shopee',id=str(items_id[0]))
print(list(res['_source'].keys())[:3])
print(res['_source']['item_itemid'])

In [ ]:
# res=es.delete(index='shopee',id=str(items_id[0]))
res=es.delete(index='shopee',id=str(_id))

In [ ]:
res= es.search(
    index='shopee',
    body={'query':{'match_all':{}}}
)
print(len(res['hits']['hits']))

In [ ]:
for i in res['hits']['hits']:
    es.delete(index=i['_index'], id=i['_id'])

# json requests

In [ ]:
json = {
    "query":"quần áo",
    "maxPrice": 600000000,
    "minPrice": 0,
    "priceOrder": "desc",
    "page": 0,
    "sortBy": "sales" #[sales -> bán chạy, price, ctime -> mới]
}

In [ ]:
import requests
res = requests.post('http://rfantic.ddns.net:9873/search', json=json)
print(res.json())

In [ ]:
print(res.request.body)
print(res.request.headers)

# Search

In [ ]:
from elasticsearch_dsl import Search
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Q
# from elasticsearch_dsl.query import MultiMatch, Match
es=Elasticsearch([{'host':'localhost','port':9200}])

In [ ]:
# m = MultiMatch(query='quần áo', fields=['item_name', 'item_description'])

In [242]:
s = Search(using=es, index="shopee") \
#     .query("match", item_name="*quần*")   \
#     .query("match", item_itemid="811146902")   \
#     .filter("term", category="search") \
#     .exclude("match", description="beta")

# s.aggs.bucket('per_tag', 'terms', field='tags') \
#     .metric('max_lines', 'max', field='lines')

# s = s.sort({"item_ctime" : {"order" : "desc"}})
# item_ctime, item_price, item_stock, item_sold
q = Q("multi_match", query='áo chốg', fields=['item_name', 'item_name.folded'], fuzziness='auto') #, 'item_description'])
s = s.query(q)
s = s[0:20] 
response = s.execute()

In [243]:
len(response['hits']['hits'])

20

In [244]:
# [i for i in response['hits']['hits'][0]['_source']]

In [245]:
for i in range(0,5):
    print(response['hits']['hits'][i]['_source']['item_name'])

Áo Chống Nắng Nam KOJIBA Cao Cấp, Thoáng Mát, Chống Mọi Tia UV Gây Hại Cho Da
Áo Chống Nắng Nam Uniplus Cao Cấp, Chống Tia UV Gây Hại Cho Da - CN01 - Kèm Túi Pocket
[Mã WABIG19 giảm 15% đơn 150k] ❌ÁO CHỐNG NẮNG NỮ, ÁO CHỐNG NẮNG TOÀN THÂN ĐỤC LỖ . ÁO CHỐNG NẮNG DÀI ❌
[Hàng Xịn] Áo Chống Nắng Vải Thông Hơi, Tản Nhiệt Cực Mát, Cho Cả Nam & Nữ
Váy Tĩnh Điện Chống Tĩnh Điện Cho Nữ


# Get index _

In [ ]:
from elasticsearch import Elasticsearch
es=Elasticsearch([{'host':'localhost','port':9200}])

In [ ]:
es.indices.get_field_mapping(fields=["item_name"], index="shopee_2", params=None, headers=None)

# Remove Accents

In [ ]:
es.indices.close(index="shopee", params=None, headers=None)

In [ ]:
body = {
#   "settings": {
    "analysis": {
      "analyzer": {
        "folding": {
          "tokenizer": "standard",
          "filter": [
            "lowercase",
            "asciifolding"
          ]
        }
      }
    }
#   }
}

In [ ]:
es.indices.put_settings(body, index="shopee", params=None, headers=None)

In [ ]:
body = {
    "properties": {
        "item_name": {
            "type": "text",
            "fields": {
                "folded": {
                    "type": "text",
                    "analyzer": "folding"
                }
            }
        }
    }
}

In [ ]:
es.indices.put_mapping(index="shopee", body=body, params=None, headers=None)

In [ ]:
es.indices.open(index="shopee", params=None, headers=None)

In [ ]:
body = \
{
#   "analyzer" : "folding",
  "text" : "Tôi thèm ăn thịt!",
  "field" : "item_name.folded",
#   "field" : "item_name",
}

In [ ]:
es.indices.analyze(body=body, index="shopee", params=None, headers=None)

In [ ]:
es.get(index="shopee", id="(2153259823, 27618473)", params=None, headers=None)['_source']['item_name']

# Reindex

In [ ]:
# body = \
# {
#   "source": {
#     "index": "shopee_2"
#   },
#   "dest": {
#     "index": "shopee"
#   }
# }

In [ ]:
es.reindex(body, params=None, headers=None, timeout="50m")